In [1]:
import pandas as pd

In [2]:
datos = pd.DataFrame()

In [3]:
def transform_data(df: pd.DataFrame):
    # Crear la columna de fecha
    # df['Fecha'] = pd.to_datetime(df[['ANO', 'MES', 'DIA']], format='YYYY-MM-DD')

    df = df.rename(columns={'ANO': 'year', 'MES': 'month', 'DIA': 'day'})
    
    # Crear la columna de fecha correctamente
    df['Fecha'] = pd.to_datetime(df[['year', 'month', 'day']])

    # Filtrar las columnas de horas
    columnas_horas = [col for col in df.columns if col.startswith('H')]

    # Convertir el formato a filas
    df_melted = df.melt(id_vars=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'Fecha'], 
                        value_vars=columnas_horas, 
                        var_name='Hora', 
                        value_name='Valor')

    # Convertir la columna 'Hora' a número de hora
    df_melted['Hora'] = df_melted['Hora'].str[1:].astype(int) - 1  # Eliminar 'H' y ajustar a formato de 24h
    return df_melted

In [ ]:
def create_new_data(df: pd.DataFrame):
    fecha = pd.to_datetime(df['Fecha']).agg()    
    # Crear la columna de fecha correctamente
    df['Fecha'] = pd.to_datetime(df[['year', 'month', 'day']])

    # Filtrar las columnas de horas
    columnas_horas = [col for col in df.columns if col.startswith('H')]

    # Convertir el formato a filas
    df_melted = df.melt(id_vars=['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'Fecha'], 
                        value_vars=columnas_horas, 
                        var_name='Hora', 
                        value_name='Valor')

    # Convertir la columna 'Hora' a número de hora
    df_melted['Hora'] = df_melted['Hora'].str[1:].astype(int) - 1  # Eliminar 'H' y ajustar a formato de 24h
    return df_melted

In [4]:
ruta = '../../../data/raw/madrid_data/horarios'
meses = ['ene', 'feb', 'mar', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
anhos = ['19', '20', '21', '22', '23', '24', '25']

In [ ]:
lista_dfs = []

for anho in anhos:
    for mes in meses:
        try:
            file = f'{ruta}/20{anho}/{mes}_meteo{anho}.csv'
            print(f"Procesando: {file}")
            df = pd.read_csv(file, sep=';')

            # Verificar si el DataFrame tiene datos
            if df.empty:
                print(f"⚠️ Archivo vacío: {file}")
                continue  # Saltar al siguiente archivo

            # Transformar los datos
            df_transformado = transform_data(df)

            # Verificar si la transformación devuelve un DataFrame válido
            if isinstance(df_transformado, pd.DataFrame):
                lista_dfs.append(df_transformado)  # Agregar a la lista
            else:
                print(f"⚠️ Error en la transformación del archivo: {file}")

        except Exception as e:
            if anho != '25':
                print(f"❌ ERROR al procesar {file}: {e}")
                # Intentar obtener datos del año anterior y posterior
                try:
                    anho_ant = str(int(anho) - 1).zfill(2)
                    anho_sig = str(int(anho) + 1).zfill(2)

                    file_ant = f'{ruta}/20{anho_ant}/{mes}_meteo{anho_ant}.csv'
                    file_sig = f'{ruta}/20{anho_sig}/{mes}_meteo{anho_sig}.csv'

                    df_ant = pd.read_csv(file_ant, sep=';')
                    df_sig = pd.read_csv(file_sig, sep=';')

                    if not df_ant.empty and not df_sig.empty:
                        df_ant_trans = transform_data(df_ant)
                        df_sig_trans = transform_data(df_sig)

                        df_ant_trans['']
                        
                        if isinstance(df_transformado, pd.DataFrame):
                            lista_dfs.append(df_transformado)
                        else:
                            print(f"⚠️ Error en la transformación de la media para {file}")
                    else:
                        print(f"⚠️ No se encontraron datos para calcular la media en {anho_ant} y {anho_sig}")

                except Exception as e2:
                    print(f"❌ ERROR al intentar calcular la media con años adyacentes para {file}: {e2}")

# Concatenar todos los DataFrames en uno solo
if lista_dfs:
    datos = pd.concat(lista_dfs, ignore_index=True)
else:
    print("⚠️ No se pudo cargar ningún archivo.")

Procesando: ../../../data/raw/madrid_data/horarios/2019/ene_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/feb_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/mar_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/may_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/jun_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/jul_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/ago_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/sep_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/oct_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/nov_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2019/dic_meteo19.csv
Procesando: ../../../data/raw/madrid_data/horarios/2020/ene_meteo20.csv
Procesando: ../../../data/raw/madrid_data/horarios/2020/feb_meteo20.csv
Procesando: ../../../data/raw/madrid_data/horarios/2020/mar_mete

In [6]:
datos.sort_values(by=['Hora', 'Fecha'], ascending=True, inplace=True)
datos

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,Fecha,Hora,Valor
0,28,79,102,81,28079102_81_98,2019-01-01,0,0.65
31,28,79,102,82,28079102_82_98,2019-01-01,0,64.00
62,28,79,102,83,28079102_83_98,2019-01-01,0,3.10
93,28,79,102,86,28079102_86_98,2019-01-01,0,50.00
124,28,79,102,87,28079102_87_98,2019-01-01,0,949.00
...,...,...,...,...,...,...,...,...
4120171,28,79,59,83,28079059_83_98,2025-01-31,23,3.80
4120202,28,79,59,86,28079059_86_98,2025-01-31,23,67.00
4120233,28,79,59,87,28079059_87_98,2025-01-31,23,948.00
4120264,28,79,59,88,28079059_88_98,2025-01-31,23,1.00
